In [3]:

import os
import sys

import sportsdataverse

from tqdm import tqdm


In [12]:


DATA_PATH = '../../data/'

def download_pbp_history():
    
    seasons = list(range(2006,2023))
    # since not all exist, reversing and trying until they fail
    seasons = seasons[::-1]
    seasons=[2022]
    for season in tqdm(seasons):
        mbb_df = sportsdataverse.mbb.load_mbb_pbp(seasons=[season])
        mbb_df.to_csv(os.path.join(DATA_PATH, f'ESPN/pbp/{season}.csv'), index=False)
        
    return



download_pbp_history()



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:41<00:00, 41.15s/it]


In [10]:


def download_player_boxscore_history():
    
    seasons = list(range(2003,2023))
    # since not all exist, reversing and trying until they fail
    seasons = seasons[::-1]
    seasons=[2022]
    for season in tqdm(seasons):
        mbb_df = sportsdataverse.mbb.load_mbb_player_boxscore(seasons=[season])
        mbb_df.to_csv(os.path.join(DATA_PATH, f'ESPN/player_boxscores/{season}.csv'), index=False)
        
    return



download_player_boxscore_history()


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.70s/it]


In [11]:

def download_schedules():
    
    seasons = list(range(2003,2023))
    # since not all exist, reversing and trying until they fail
    seasons = seasons[::-1]
    # for update
    seasons=[2022]
    for season in tqdm(seasons):
        mbb_df = sportsdataverse.mbb.load_mbb_schedule(seasons=[season])
        mbb_df.to_csv(os.path.join(DATA_PATH, f'ESPN/schedules/by_season/{season}.csv'), index=False)
        
    return

download_schedules()


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.57it/s]


In [13]:

# mbb_df = sportsdataverse.mbb.load_mbb_team_boxscore(seasons=range(2002,2023))


def download_team_boxscore_history():
    
    seasons = list(range(2003,2023))
    # since not all exist, reversing and trying until they fail
    seasons = seasons[::-1]
#     seasons=[2022]
    for season in tqdm(seasons):
        mbb_df = sportsdataverse.mbb.load_mbb_team_boxscore(seasons=[season])
        mbb_df.to_csv(os.path.join(DATA_PATH, f'ESPN/team_boxscores/{season}.csv'), index=False)
        
    return

download_team_boxscore_history()



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:16<00:00,  1.24it/s]
